# HIMALAYAS

### In brief...

### For every iteration, the df is inspected and then filtered by its pubDate (if there's one). Finally, it creates a new df that will be transformed into a new table in the local postgreSQL DB.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import mysql.connector
import psycopg2
import numpy as np
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse

In [2]:
df = pd.read_csv('/Users/juanreyesgarcia/Library/CloudStorage/OneDrive-FundacionUniversidaddelasAmericasPuebla/DEVELOPER/PROJECTS/CRAWLER_ALL/himalaya.csv')

##### Play with the settings...

In [3]:
pd.set_option('display.max_colwidth', 150)

In [4]:
pd.set_option("display.max_rows", None)

# EDA Checklist 

In [5]:
df[:100]

,title,link,pubDate,location,description
0,"['Mgr., Support Engineering, Marketplace Solutions']",https://himalayas.app/companies/platform-science/jobs/mgr-support-engineering-marketplace-solutions,55 minutes ago,United States only,Support Engineer
1,['Growth Marketing Manager'],https://himalayas.app/companies/grow-therapy/jobs/growth-marketing-manager,2 hours ago,United States only,Marketing Manager
2,['Technical Support Engineer (German Speaking) - EMEA Remote'],https://himalayas.app/companies/degreed/jobs/technical-support-engineer-german-speaking-emea-remote,2 hours ago,"EMEA, CH + 5 more only",Technical Support Engineer
3,['Staff Security Engineer'],https://himalayas.app/companies/grow-therapy/jobs/staff-security-engineer,2 hours ago,United States only,Security Engineer
4,['Senior NetSuite Developer'],https://himalayas.app/companies/pleo/jobs/senior-netsuite-developer-6658170280,3 hours ago,ES &amp; PT,Software Engineer
5,['Revenue Accounting Manager'],https://himalayas.app/companies/platform-science/jobs/revenue-accounting-manager-7787104513,4 hours ago,United States only,Accounting Manager
6,['Senior Animator'],https://himalayas.app/companies/classdojo/jobs/senior-animator,4 hours ago,United States only,Animator
7,['Data Analyst'],https://himalayas.app/companies/creatopy/jobs/data-analyst-5889230599,6 hours ago,United States only,Data Scientist
8,['Senior Product Manager - Snyk Code Group'],https://himalayas.app/companies/snyk/jobs/senior-product-manager-snyk-code-group-843237655,7 hours ago,Canada only,Product Manager
9,['Senior Software Development Engineer in Test'],https://himalayas.app/companies/degreed/jobs/senior-software-development-engineer-in-test,8 hours ago,United States only,Software Engineer


In [6]:
df.tail()

,title,link,pubDate,location,description
2012,['Senior Software Engineer (Fullstack)'],https://himalayas.app/companies/abridge/jobs/senior-software-engineer-fullstack-2540980473,1 month ago,NaN,Product Designer
2013,['Senior Scala Backend Engineer'],https://himalayas.app/companies/goodcover/jobs/senior-scala-backend-engineer,1 month ago,NaN,Software Engineer
2014,['Senior Product Designer'],https://himalayas.app/companies/elevate/jobs/senior-product-designer,1 month ago,NaN,Front End Developer
2015,['Intern - Web Developer/ Video Tutorial Creator'],https://himalayas.app/companies/blockonomics/jobs/head-of-product-design,4 months ago,NaN,NaN
2016,['Senior Front-end Developer'],https://himalayas.app/companies/occtoo/jobs/front-end-developer,5 months ago,NaN,NaN


### Describe the DF (current)

In [8]:
df.describe()

,title,link,pubDate,location,description
count,2017,2017,2017,1688,2015
unique,1576,2017,46,107,373
top,['Senior Software Engineer'],https://himalayas.app/companies/platform-science/jobs/mgr-support-engineering-marketplace-solutions,1 month ago,United States only,Software Engineer
freq,29,1,906,971,363


### Checking for missing values DF (current)

In [9]:
df.isnull().sum()

title            0
link             0
pubDate          0
location       329
description      2
dtype: int64

### Fill missing values with "NaN" DF (current)

In [10]:
# Fill missing values with "NaN"
df.fillna("NaN", inplace=True)

In [11]:
df.isnull().sum()

title          0
link           0
pubDate        0
location       0
description    0
dtype: int64

### Checking for duplicates DF (current)

In [12]:
df.duplicated().sum()

0

In [13]:
df.describe()

,title,link,pubDate,location,description
count,2017,2017,2017,2017,2017
unique,1576,2017,46,108,374
top,['Senior Software Engineer'],https://himalayas.app/companies/platform-science/jobs/mgr-support-engineering-marketplace-solutions,1 month ago,United States only,Software Engineer
freq,29,1,906,971,363


In [14]:
df.dtypes

title          object
link           object
pubDate        object
location       object
description    object
dtype: object

### Mask to convert %minutes|hours to 1 day ago so it can be parse to date time (cba to say it was posted today, maybe later should be fixed)

In [15]:
mask = df['pubDate'].str.contains('minutes|hours')

In [16]:
df.loc[mask, 'pubDate'] = "1 day ago"

In [17]:
df.head()

,title,link,pubDate,location,description
0,"['Mgr., Support Engineering, Marketplace Solutions']",https://himalayas.app/companies/platform-science/jobs/mgr-support-engineering-marketplace-solutions,1 day ago,United States only,Support Engineer
1,['Growth Marketing Manager'],https://himalayas.app/companies/grow-therapy/jobs/growth-marketing-manager,1 day ago,United States only,Marketing Manager
2,['Technical Support Engineer (German Speaking) - EMEA Remote'],https://himalayas.app/companies/degreed/jobs/technical-support-engineer-german-speaking-emea-remote,1 day ago,"EMEA, CH + 5 more only",Technical Support Engineer
3,['Staff Security Engineer'],https://himalayas.app/companies/grow-therapy/jobs/staff-security-engineer,1 day ago,United States only,Security Engineer
4,['Senior NetSuite Developer'],https://himalayas.app/companies/pleo/jobs/senior-netsuite-developer-6658170280,1 day ago,ES &amp; PT,Software Engineer


# From relative date strings to date time...

In [18]:
# create a mask to identify rows with relative date strings
mask = df['pubDate'].str.contains('ago')

# apply the relative date calculation to the date_string column
df.loc[mask, 'datetime'] = pd.Timestamp.today() - df.loc[mask, 'pubDate'].apply(lambda x: relativedelta(days=int(x.split()[0])))

# convert the datetime column to datetime format
df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True)


/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_95487/1559547877.py:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[mask, 'datetime'] = pd.Timestamp.today() - df.loc[mask, 'pubDate'].apply(lambda x: relativedelta(days=int(x.split()[0])))


In [19]:
df.head(50)

,title,link,pubDate,location,description,datetime
0,"['Mgr., Support Engineering, Marketplace Solutions']",https://himalayas.app/companies/platform-science/jobs/mgr-support-engineering-marketplace-solutions,1 day ago,United States only,Support Engineer,2023-03-04 20:30:40.568613
1,['Growth Marketing Manager'],https://himalayas.app/companies/grow-therapy/jobs/growth-marketing-manager,1 day ago,United States only,Marketing Manager,2023-03-04 20:30:40.568613
2,['Technical Support Engineer (German Speaking) - EMEA Remote'],https://himalayas.app/companies/degreed/jobs/technical-support-engineer-german-speaking-emea-remote,1 day ago,"EMEA, CH + 5 more only",Technical Support Engineer,2023-03-04 20:30:40.568613
3,['Staff Security Engineer'],https://himalayas.app/companies/grow-therapy/jobs/staff-security-engineer,1 day ago,United States only,Security Engineer,2023-03-04 20:30:40.568613
4,['Senior NetSuite Developer'],https://himalayas.app/companies/pleo/jobs/senior-netsuite-developer-6658170280,1 day ago,ES &amp; PT,Software Engineer,2023-03-04 20:30:40.568613
5,['Revenue Accounting Manager'],https://himalayas.app/companies/platform-science/jobs/revenue-accounting-manager-7787104513,1 day ago,United States only,Accounting Manager,2023-03-04 20:30:40.568613
6,['Senior Animator'],https://himalayas.app/companies/classdojo/jobs/senior-animator,1 day ago,United States only,Animator,2023-03-04 20:30:40.568613
7,['Data Analyst'],https://himalayas.app/companies/creatopy/jobs/data-analyst-5889230599,1 day ago,United States only,Data Scientist,2023-03-04 20:30:40.568613
8,['Senior Product Manager - Snyk Code Group'],https://himalayas.app/companies/snyk/jobs/senior-product-manager-snyk-code-group-843237655,1 day ago,Canada only,Product Manager,2023-03-04 20:30:40.568613
9,['Senior Software Development Engineer in Test'],https://himalayas.app/companies/degreed/jobs/senior-software-development-engineer-in-test,1 day ago,United States only,Software Engineer,2023-03-04 20:30:40.568613


### Drop pubDate, change name of datetime & reindex...

In [20]:
#Drop it
df = df.drop('pubDate', axis=1)

In [23]:
#Change name of datetime to pubDate 
df = df.rename(columns={'datetime': 'pubDate'})

In [25]:
#reindex
df = df.reindex(columns=['title', 'link', 'description', 'pubDate', 'location'])

In [26]:
df.head()

,title,link,description,pubDate,location
0,"['Mgr., Support Engineering, Marketplace Solutions']",https://himalayas.app/companies/platform-science/jobs/mgr-support-engineering-marketplace-solutions,Support Engineer,2023-03-04 20:30:40.568613,United States only
1,['Growth Marketing Manager'],https://himalayas.app/companies/grow-therapy/jobs/growth-marketing-manager,Marketing Manager,2023-03-04 20:30:40.568613,United States only
2,['Technical Support Engineer (German Speaking) - EMEA Remote'],https://himalayas.app/companies/degreed/jobs/technical-support-engineer-german-speaking-emea-remote,Technical Support Engineer,2023-03-04 20:30:40.568613,"EMEA, CH + 5 more only"
3,['Staff Security Engineer'],https://himalayas.app/companies/grow-therapy/jobs/staff-security-engineer,Security Engineer,2023-03-04 20:30:40.568613,United States only
4,['Senior NetSuite Developer'],https://himalayas.app/companies/pleo/jobs/senior-netsuite-developer-6658170280,Software Engineer,2023-03-04 20:30:40.568613,ES &amp; PT


## Filter rows by a date range (this reduces the number of rows... duh)

In [27]:
# filter rows by a date range
start_date = pd.to_datetime('2016-01-01')
end_date = pd.to_datetime('2023-02-15')

#for df
date_range_filter = (df['pubDate'] >= start_date) & (df['pubDate'] <= end_date)
df = df.loc[~date_range_filter]

In [28]:
df = df.sort_values(by='pubDate')

In [29]:
df.describe(datetime_is_numeric=True)

,pubDate
count,1432
mean,2023-03-01 23:40:43.920568576
min,2023-02-15 20:30:40.568613
25%,2023-02-28 20:30:40.568613120
50%,2023-03-04 20:30:40.568613120
75%,2023-03-04 20:30:40.568613120
max,2023-03-04 20:30:40.568613


In [31]:
df.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_95487/3627053830.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


,title,link,description,pubDate,location
count,1432,1432,1432,1432,1432
unique,1144,1432,311,17,87
top,['Senior Software Engineer'],https://himalayas.app/companies/kajabi/jobs/lead-generation-manager,Software Engineer,2023-03-04 20:30:40.568613,United States only
freq,20,1,243,949,649
first,NaN,NaN,NaN,2023-02-15 20:30:40.568613,NaN
last,NaN,NaN,NaN,2023-03-04 20:30:40.568613,NaN


In [32]:
df.dtypes

title                  object
link                   object
description            object
pubDate        datetime64[ns]
location               object
dtype: object

### Make a copy just in case

In [35]:
df1 = df.copy()

In [33]:
# replace NaT values in the DataFrame with None
df = df.replace({np.nan: None, pd.NaT: None})

In [34]:
df

,title,link,description,pubDate,location
504,['Lead Generation Manager'],https://himalayas.app/companies/kajabi/jobs/lead-generation-manager,Software Engineer,2023-02-15 20:30:40.568613,North America only
505,['Lead Full Stack Software Engineer'],https://himalayas.app/companies/mozilla/jobs/lead-full-stack-software-engineer,Product Manager,2023-02-15 20:30:40.568613,Germany only
506,['Principal Software Engineer'],https://himalayas.app/companies/nuna/jobs/principal-software-engineer-2538228308,Analytics Manager,2023-02-15 20:30:40.568613,"US, CA &amp; LATAM"
507,['Senior Product Manager'],https://himalayas.app/companies/headway-co/jobs/senior-product-manager,Director of Marketing,2023-02-15 20:30:40.568613,India only
508,['Health Data Analytics Engineer'],https://himalayas.app/companies/nuna/jobs/health-data-analytics-engineer-9362566542,Software Engineer,2023-02-15 20:30:40.568613,United States only
509,"['Director, Performance Marketing']",https://himalayas.app/companies/kajabi/jobs/director-performance-marketing,Content Designer,2023-02-15 20:30:40.568613,United States only
511,['Senior Content Designer'],https://himalayas.app/companies/upwork/jobs/senior-content-designer,Account Executive,2023-02-15 20:30:40.568613,United States only
512,['Director of RCM-Finance'],https://himalayas.app/companies/medarrive/jobs/director-of-rcm-finance,Lawyer,2023-02-15 20:30:40.568613,United States only
513,['Enterprise Account Executive DACH'],https://himalayas.app/companies/algolia/jobs/enterprise-account-executive-dach,Engineering Manager,2023-02-15 20:30:40.568613,US &amp; CA
514,"['Senior Legal Counsel - Employment, Incentives &amp; Pensions']",https://himalayas.app/companies/monzo/jobs/senior-legal-counsel-employment-incentives-pensions,Technical Support Engineer,2023-02-15 20:30:40.568613,"DE, ES &amp; GB"


# PostgreSQL

### This code creates a new table per iteration

In [36]:
# create a connection to the PostgreSQL database
cnx = psycopg2.connect(user='postgres', password='3312', host='localhost', port='5432', database='postgres')

# create a cursor object
cursor = cnx.cursor()

# get the name of the next table to create
get_table_name_query = '''
    SELECT COUNT(*) FROM information_schema.tables
    WHERE table_name LIKE 'himalayas_%'
'''
cursor.execute(get_table_name_query)
result = cursor.fetchone()
next_table_number = result[0] + 1
next_table_name = 'himalayas_{}'.format(next_table_number)

# prepare the SQL query to create a new table
create_table_query = '''
    CREATE TABLE {} (
        title VARCHAR(255),
        link VARCHAR(255),
        description VARCHAR(1000),
        pubDate TIMESTAMP,
        location VARCHAR(255)
    )
'''.format(next_table_name)

# execute the create table query
cursor.execute(create_table_query)

# insert the DataFrame into the PostgreSQL database as a new table
for index, row in df.iterrows():
    insert_query = '''
        INSERT INTO {} (title, link, description, pubDate, location)
        VALUES (%s, %s, %s, %s, %s)
    '''.format(next_table_name)
    values = (row['title'], row['link'], row['description'], row['pubDate'], row['location'])
    cursor.execute(insert_query, values)

# commit the changes to the database
cnx.commit()

# close the cursor and connection
cursor.close()
cnx.close()